**Step 1: Load Data**

please remember to upload the file "ITC.NS_historical_data.csv"

In [81]:
import pandas as pd
import numpy as np

# File path in Google Colab environment
file_path = 'content/ITC.NS_historical_data.csv'
# file_path = 'content/SyntheticTC/1.csv'
# File path for Anil's Physical computer
# file_path = 'D:\DKafka Admin\Outsource\VCP\HistoricalData\ITC.NS_historical_data.csv'

df = pd.read_csv(file_path)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Adding moving averages to the DataFrame with min_periods=1
df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()
df['MA150'] = df['Close'].rolling(window=150, min_periods=1).mean()
df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()

# Display the first few rows to ensure it's loaded correctly
print(df.head())


        Date      Open      High       Low     Close  Adj Close    Volume  \
0 1996-01-01  5.550000  5.600000  5.533333  5.583333   3.255736    985500   
1 1996-01-02  5.466666  5.566666  5.288888  5.372222   3.132634   7470000   
2 1996-01-03  5.133333  5.254444  5.101111  5.200000   3.032208  15160500   
3 1996-01-04  5.200000  5.332222  5.144444  5.297777   3.089224  12397500   
4 1996-01-05  5.297777  5.277777  5.188888  5.202222   3.033503   5008500   

      MA200     MA150      MA50  
0  5.583333  5.583333  5.583333  
1  5.477777  5.477777  5.477777  
2  5.385185  5.385185  5.385185  
3  5.363333  5.363333  5.363333  
4  5.331111  5.331111  5.331111  


**Step 2: Filter data for last 3 months**

In [82]:
# Assuming 'df' is your DataFrame and it's already loaded and 'Date' is in datetime format
pd.set_option('display.max_rows', None)

# Get the last date in the dataset
last_date = df['Date'].max()

# Calculate the date 3 months prior to the last date
three_months_ago = last_date - pd.DateOffset(months=150)

# Filter the DataFrame for the last 3 months
df_last_3_months = df[df['Date'] > three_months_ago]
# df_last_3_months = df_last_3_months[:-1]
# data = {
#         'Date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
#         'Open': [100, 105, 110, 108, 112, 115, 114, 118, 120, 122],
#         'High': [110, 115, 120, 118, 122, 125, 124, 128, 130, 132],
#         'Low': [98, 103, 108, 106, 110, 113, 112, 116, 118, 120],
#         'Close': [105, 110, 115, 113, 118, 120, 119, 123, 125, 128]
#     }

# df_last_3_months=pd.DataFrame(data)
# df_last_3_months['MA200'] = df_last_3_months['Close'].rolling(window=200, min_periods=1).mean()
# df_last_3_months['MA150'] = df_last_3_months['Close'].rolling(window=150, min_periods=1).mean()
# df_last_3_months['MA50'] = df_last_3_months['Close'].rolling(window=50, min_periods=1).mean()

last_date = df['Date'].max()
three_months_ago = last_date - pd.DateOffset(months=1)
df_last_3_months = df[df['Date'] > three_months_ago]
# print(df_last_3_months)  # Just to verify the filtering


In [83]:
import plotly.graph_objects as go

# Create a candlestick chart for the last 3 months
fig = go.Figure(data=[go.Candlestick(x=df_last_3_months['Date'],
                                     open=df_last_3_months['Open'],
                                     high=df_last_3_months['High'],
                                     low=df_last_3_months['Low'],
                                     close=df_last_3_months['Close'], name="Candlestick")])

# Add MA200
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA200'],
                         mode='lines', name='MA200',
                         line=dict(width=1, color='blue')))

# Add MA150
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA150'],
                         mode='lines', name='MA150',
                         line=dict(width=1, color='red')))

# Add MA50
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA50'],
                         mode='lines', name='MA50',
                         line=dict(width=1, color='green')))

# Customize the layout
fig.update_layout(title='ITC.NS Historical Data (Last 3 Months) with Moving Averages',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)  # Hide the range slider for a cleaner look

# Show the plot
fig.show()


# **Utility Functions**

---

**Candle diff**

Purpose of the function is to find the number of candles between two candles.

**Limitation** >>  Can't be used for intraday candles

In [84]:
def candle_diff(date1, date2, df):
    """
    Calculates the number of candles (rows) between two given candle dates in a DataFrame.

    Parameters:
    - date1: The date of the first candle (string or datetime).
    - date2: The date of the second candle (string or datetime).
    - df: DataFrame containing the OHLC data with a 'Date' column.

    Returns:
    - The number of candles between the two given dates, excluding non-trading days.
    """
    # Convert date1 and date2 to datetime if they are not already
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)

    # Ensure date1 is before date2
    if date1 > date2:
        date1, date2 = date2, date1  # Swap the dates if date1 is later than date2

    # Find the indices for the given dates
    index1 = df.index[df['Date'] == date1].tolist()
    index2 = df.index[df['Date'] == date2].tolist()

    # Check if both dates exist in the DataFrame
    if not index1 or not index2:
        return None  # One or both of the dates do not exist in the DataFrame

    # Calculate the difference in indices to find the number of candles between the dates
    candle_count = index2[0] - index1[0]

    return candle_count


In [85]:
# Sample usage:
# Assuming df is your DataFrame containing the OHLC data
date1 = '2024-01-19'
date2 = '2024-01-22'
diff = candle_diff(date1, date2, df)
print(f"Number of candles between {date1} and {date2}: {diff}")


Number of candles between 2024-01-19 and 2024-01-22: 1


**get_index**


purpose to get the index of the candle for given date.   This is for easy testing and not to be used in prod.

In [86]:
def get_index(candle_date, df):
    """
    Find the index of the candle with a given date in the DataFrame. Stops processing if the date is not found.

    Parameters:
    - df: DataFrame containing the OHLC data with a 'Date' column.
    - candle_date: The date of the candle to find, as a string in the same format as the DataFrame's 'Date' column.

    Returns:
    - The index of the candle with the given date. Halts execution with an assertion error if the date is not found.
    """
    # Convert the 'Date' column to datetime if it's not already
    df['Date'] = pd.to_datetime(df['Date'])

    # Convert the input date to datetime
    candle_date = pd.to_datetime(candle_date)

    # Find the index of the row that matches the given date
    match = df.index[df['Date'] == candle_date].tolist()

    # Assert that the match list is not empty; if it is, halt execution
    assert match, "Date not found in given OHLC DataFrame. Processing halted."

    # Return the index if found
    return match[0]

# Example usage:
# df = pd.read_csv('path_to_your_file.csv')  # Load your DataFrame here
# candle_index = get_index('1996-01-01', df)
# print(candle_index)


In [87]:
print(get_index('2023-12-08', df_last_3_months))
print(get_index('1996-01-01', df))
print(type(get_index('2011-08-09', df_last_3_months)))

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



AssertionError: Date not found in given OHLC DataFrame. Processing halted.

In [ ]:
df_last_3_months.index.min()

3981

In [ ]:
df.index.max()

7061

**get_candle_date**

get candle date for given index

In [ ]:
def get_candle_date(df, index):
    """
    Retrieves the date of the candle for a given index from the DataFrame.

    Parameters:
    - df: DataFrame containing the OHLC data with a 'Date' column.
    - index: The index of the candle whose date you want to retrieve.

    Returns:
    - The date of the candle at the given index, or None if the index is not found.
    """
    try:
        # Access the 'Date' column of the DataFrame at the given index
        candle_date = df.loc[index, 'Date']
        return candle_date
    except KeyError:
        # Return None if the index is not found
        return None


**Is engulfing next Function**

The overarching idea over here is to find out how many subsequent candles are completely engulfed by a preceding candle.  A candle say A is said to engulfing the subsequent candle say B,  if the high of B is lower than high of A and low of B is higher than low of A.  We need to see how many subsequent candles are being engulfed by a given candle.  Answer might be 0, 1, 2 or n depending on the candle data.   The logic for this function is given below.

In [ ]:
def engulfed_candle_count(candle_index, df):
    """
    Count how many subsequent candles are engulfed by the candle at a given index.

    Parameters:
    - candle_index: The index of the candle to start the check from.
    - df: DataFrame containing the OHLC data.

    Returns:
    - The number of subsequent candles engulfed by the candle at the given index.
    """
    # Initialize the count of engulfed candles
    engulfed_candle_count = 0

    # Find the maximum index in the DataFrame
    max_index = df.index.max()

    # Start a loop to check each subsequent candle
    while True:
        # Ensure we do not exceed the maximum index of the DataFrame
        if candle_index + engulfed_candle_count + 1 > max_index:
            break

        # Get the current and next candle data
        current_candle = df.loc[candle_index]
        next_candle_index = df.index[df.index > candle_index][engulfed_candle_count]  # Get the next available index
        if next_candle_index > max_index:
            break  # Break if the next index is beyond the DataFrame's range

        next_candle = df.loc[next_candle_index]

        # Check if the current candle engulfs the next candle
        if current_candle['High'] >= next_candle['High'] and current_candle['Low'] <= next_candle['Low']:
            # Increase the count as the next candle is engulfed
            engulfed_candle_count += 1
        else:
            # If the next candle is not engulfed, break the loop
            break

    # Return the count of engulfed candles
    return engulfed_candle_count


**Testing Engulfed_candle_count_function**

In [ ]:
print("No of candles engulfed @ 2024-01-30 are : ", engulfed_candle_count(get_index('2024-01-30',df_last_3_months), df_last_3_months))
print("No of candles engulfed @ 2023-11-16 are : ", engulfed_candle_count(get_index('2023-11-16',df_last_3_months), df_last_3_months))

# checking for second last
print("No of candles engulfed @ 2024-02-07 are : ", engulfed_candle_count(get_index('2024-02-07',df_last_3_months), df_last_3_months))

print("Number of candles @ 2024-01-19 engulfed are : ", engulfed_candle_count(get_index('2024-01-19',df_last_3_months), df_last_3_months))

print("Number of candles @ 2023-12-08 engulfed are : ", engulfed_candle_count(get_index('2023-12-08',df_last_3_months), df_last_3_months))

No of candles engulfed @ 2024-01-30 are :  3
No of candles engulfed @ 2023-11-16 are :  1
No of candles engulfed @ 2024-02-07 are :  0
Number of candles @ 2024-01-19 engulfed are :  1
Number of candles @ 2023-12-08 engulfed are :  4


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

**Candle_from_date**


In [ ]:
def candle_from_date(candle_date, df):
    """
    Utility function to get the candle data for a specific date from the DataFrame.

    Parameters:
    - candle_date: The date of the candle to retrieve.
    - df: DataFrame containing OHLC data with a 'Date' column.

    Returns:
    - The row from the DataFrame corresponding to the candle_date.
    """
    return df.loc[df['Date'] == pd.to_datetime(candle_date)].iloc[0]


**candle_from_extreme_point**


In [ ]:
def candle_from_extreme_point(extreme_point, df):
    """
    Retrieves a candle from the DataFrame based on the date in an extreme point dictionary.

    Parameters:
    - extreme_point: Dictionary with 'mark type', 'date', and 'value' keys.
    - df: DataFrame containing OHLC data with a 'Date' column.

    Returns:
    - The candle data corresponding to the date in the extreme_point dictionary.
    """
    # If extreme_point is None then candle would also be none
    if extreme_point is None:
        return None

    # Extract the date from the extreme_point dictionary
    extreme_point_date = pd.to_datetime(extreme_point['date']).date()

    # Use the candle_from_date utility function to get the candle data
    return candle_from_date(extreme_point_date, df)


In [ ]:
extreme_point = {
    'mark type': 'max',
    'date': pd.Timestamp('2024-01-04 00:00:00'),
    'value': 481.4500122070313
}

# Retrieve the candle corresponding to the extreme point
candle_data = candle_from_extreme_point(extreme_point, df)
print(candle_data)


Date         2024-01-04 00:00:00
Open                       480.0
High                  481.450012
Low                   474.200012
Close                 476.399994
Adj Close             469.504456
Volume                  11786959
MA200                 439.200001
MA150                 450.213001
MA50                  446.436002
Name: 7037, dtype: object


# **Extreme Points ZigZag New Approach**
---

To implement threshold based approach right at the outset..

**Step 1: NEXT_EXTREME_HIGH_POINT**

In [ ]:
def next_extreme_high_point(df, start_candle_index=None, input_candle_index=None, percent_threshold=0, move_relative_percent_threshold=0, absolute_threshold=0, pivot_high_candle_index=-1):
    """
    Finds the next extreme high point in stock data, filtering out minor movements below a specified threshold.
    Enhanced with logging for debugging purposes.
    """
    # Initialization with logging
    if start_candle_index is None:
        start_candle_index = df.index.min()
    if input_candle_index is None:
        input_candle_index = start_candle_index
    if pivot_high_candle_index == -1:
        pivot_high_candle_index = input_candle_index

    # print(f"Starting search from index: {start_candle_index}, Input candle index: {input_candle_index}, Pivot high index: {pivot_high_candle_index}")

    pivot_high = df.loc[pivot_high_candle_index, 'High']
    # print(f"Initial pivot high: {pivot_high} at index {pivot_high_candle_index}")

    engulfed_count = engulfed_candle_count(input_candle_index, df)
    next_candle_index = input_candle_index + engulfed_count + 1

    if next_candle_index > df.index.max():
        print("Reached the end of the DataFrame. No more candles to evaluate.")
        return (pivot_high_candle_index, pivot_high, "HIGH")

    next_candle = df.loc[next_candle_index]
    # print(f"Evaluating next candle at index {next_candle_index} with High: {next_candle['High']}")

    if next_candle['High'] > pivot_high:
        pivot_high = next_candle['High']
        pivot_high_candle_index = next_candle_index
        print(f"New pivot high found: {pivot_high} at index {pivot_high_candle_index}")
    else:
        move_in_high = pivot_high - next_candle['High']
        percent_high_move_threshold = pivot_high * (percent_threshold / 100)
        last_move = pivot_high - df.loc[start_candle_index, 'High']
        last_move_based_threshold = last_move * move_relative_percent_threshold /100
        high_move_abs_threshold = max(absolute_threshold, percent_high_move_threshold, last_move_based_threshold)
        # print(f"Move in high: {move_in_high}, Last Move = {last_move}, Last Move based Threshold:{last_move_based_threshold}, Percent high move threshold: {percent_high_move_threshold}, Absolute threshold: {absolute_threshold}")

        if move_in_high >= high_move_abs_threshold:
            print(f"Threshold met. Returning pivot high index: {pivot_high_candle_index}")
            return (pivot_high_candle_index, pivot_high, "HIGH")

    # print("Found new pivot high  OR  Threshold not met. Either ways Continuing search...")
    return next_extreme_high_point(df, start_candle_index=start_candle_index, input_candle_index=next_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold, absolute_threshold=absolute_threshold, pivot_high_candle_index=pivot_high_candle_index)


In [ ]:
# Make sure df_last_3_months['Date'] is in datetime format
df_last_3_months['Date'] = pd.to_datetime(df_last_3_months['Date'])

# Find the index for December 21, 2023
start_date = '2023-11-01'
start_candle_index = get_index(start_date,  df_last_3_months)
next_high_index, next_high_value, extreme_type = next_extreme_high_point(df_last_3_months, start_candle_index=start_candle_index, percent_threshold=3, move_relative_percent_threshold=30, absolute_threshold=0)
print(f"Next extreme {extreme_type} > {next_high_value} is on date:", get_candle_date(df_last_3_months, next_high_index))

New pivot high found: 432.7000122070313 at index 6995
New pivot high found: 434.3999938964844 at index 6996
New pivot high found: 436.7999877929688 at index 6997
New pivot high found: 437.5 at index 6999
New pivot high found: 438.0 at index 7000
New pivot high found: 438.3999938964844 at index 7002
New pivot high found: 442.5 at index 7003
New pivot high found: 444.5 at index 7004
New pivot high found: 450.9500122070313 at index 7014
New pivot high found: 456.3999938964844 at index 7015
New pivot high found: 457.7999877929688 at index 7016
New pivot high found: 464.5 at index 7017
New pivot high found: 464.7999877929688 at index 7018
New pivot high found: 467.8999938964844 at index 7033
New pivot high found: 469.9500122070313 at index 7034
New pivot high found: 471.3500061035156 at index 7035
New pivot high found: 480.7000122070313 at index 7036
New pivot high found: 481.4500122070313 at index 7037
Threshold met. Returning pivot high index: 7037
Next extreme HIGH > 481.4500122070313 is

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1622473385.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Step 2: NEXT_EXTREME_LOW_POINT**

In [ ]:
def next_extreme_low_point(df, start_candle_index=None, input_candle_index=None, percent_threshold=0, move_relative_percent_threshold=0, absolute_threshold=0, pivot_low_candle_index=-1):
    """
    Identifies the next extreme low point in a dataset of stock prices, filtering out minor movements.
    Debugging statements are active for detailed insights during execution.
    """
    if start_candle_index is None:
        start_candle_index = df.index.min()

    if input_candle_index is None:
        input_candle_index = start_candle_index

    if pivot_low_candle_index == -1:
        pivot_low_candle_index = input_candle_index

    pivot_low = df.loc[pivot_low_candle_index, 'Low']
    # print(f"Starting search from index: {start_candle_index}, Input candle index: {input_candle_index}, Pivot low index: {pivot_low_candle_index}, Initial pivot low: {pivot_low}")

    engulfed_count = engulfed_candle_count(input_candle_index, df)
    next_candle_index = input_candle_index + engulfed_count + 1

    if next_candle_index > df.index.max():
        # print(f"Reached the end of the DataFrame. No more candles to evaluate.End index of df is {df.index.max()}")
        # print("pivot_low_candle_index is  ", pivot_low_candle_index)
        return (pivot_low_candle_index, pivot_low, "LOW")

    next_candle = df.loc[next_candle_index]
    # print(f"Evaluating next candle at index {next_candle_index} with Low: {next_candle['Low']}")

    if next_candle_index > df.index.max():
        print("Reached the end of the DataFrame. No more candles to evaluate.")
        return (pivot_low_candle_index, pivot_low, "LOW")

    if next_candle['Low'] < pivot_low:
        pivot_low = next_candle['Low']
        pivot_low_candle_index = next_candle_index
        print(f"New pivot low found: {pivot_low} at index {pivot_low_candle_index}")
    else:
        move_in_low = next_candle['Low'] - pivot_low
        percent_low_move_threshold = pivot_low * (percent_threshold / 100)
        last_move = df.loc[start_candle_index, 'Low'] - pivot_low
        last_move_based_threshold = last_move * move_relative_percent_threshold /100
        low_move_abs_threshold = max(absolute_threshold, percent_low_move_threshold, last_move_based_threshold)
        # print(f"Move in low: {move_in_low}, Last Move = {last_move}, Last Move based Threshold:{last_move_based_threshold}, Percent low move threshold: {percent_low_move_threshold}, Absolute threshold: {low_move_abs_threshold}")

        if move_in_low >= low_move_abs_threshold:  # Check for significant move away from the low
            print(f"Threshold met. Returning pivot low index: {pivot_low_candle_index}")
            return (pivot_low_candle_index, pivot_low, "LOW")

    # print("Found new pivot low  OR  Threshold not met. Either ways Continuing search...")
    return next_extreme_low_point(df, start_candle_index=start_candle_index, input_candle_index=next_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold, absolute_threshold=absolute_threshold, pivot_low_candle_index=pivot_low_candle_index)


In [ ]:
# Make sure df_last_3_months['Date'] is in datetime format
df_last_3_months['Date'] = pd.to_datetime(df_last_3_months['Date'])

# Find the index for December 21, 2023
start_date = '2023-01-04'
start_candle_index = get_index(start_date,  df_last_3_months)

next_low_index, next_low_point, extreme_type = next_extreme_low_point(df_last_3_months, start_candle_index=start_candle_index, percent_threshold=2, move_relative_percent_threshold=20, absolute_threshold=0)
print(f"Next extreme {extreme_type} point > {next_low_point} is at date:", get_candle_date(df_last_3_months, next_low_index))



C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\445644996.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Threshold met. Returning pivot low index: 6791
Next extreme LOW point > 326.0 is at date: 2023-01-04 00:00:00


**Step 3: NEXT_EXTREME_CANDLE**

In [ ]:
def next_extreme_candle(df, start_candle_index, input_candle_index=None, percent_threshold=0, move_relative_percent_threshold=0, absolute_threshold=0, pivot_high_candle_index=-1, pivot_low_candle_index=-1):
    """
    Identifies the next extreme point (high or low) in stock price data, filtering out minor fluctuations.

    Parameters:
    - df: DataFrame containing stock data.
    - start_candle_index: Index to start the search from.
    - input_candle_index: Current candle being considered. If None, set to start_candle_index.
    - percent_threshold: Minimum percentage movement required to consider a point significant.
    - absolute_threshold: Minimum absolute movement required to consider a point significant.
    - pivot_high_candle_index: Index of the current pivot high candle. Initialized to start_candle_index if -1.
    - pivot_low_candle_index: Index of the current pivot low candle. Initialized to start_candle_index if -1.

    Returns:
    - Index of the next extreme point that meets threshold criteria.
    """
    if start_candle_index is None:
        start_candle_index = df.index.min()

    if input_candle_index is None:
        input_candle_index = start_candle_index

    if pivot_high_candle_index == -1 and pivot_low_candle_index == -1:
        pivot_high_candle_index = pivot_low_candle_index = input_candle_index

    pivot_high = df.loc[pivot_high_candle_index, 'High']
    pivot_low = df.loc[pivot_low_candle_index, 'Low']

    engulfed_count = engulfed_candle_count(input_candle_index, df)
    next_candle_index = input_candle_index + engulfed_count + 1

    if next_candle_index >= df.index.max():
        # print("Reached the end of the DataFrame. No further candles to evaluate. Giving most recent relevant point as default")
        if pivot_high_candle_index > pivot_low_candle_index:
              return (pivot_high_candle_index, pivot_high, "HIGH")
        elif pivot_high_candle_index < pivot_low_candle_index:
              return (pivot_low_candle_index, pivot_low, "LOW")
        else:
              return (pivot_high_candle_index, pivot_high, "HIGH")

    # print(f"next_candle_index is {next_candle_index}")
    next_candle = df.loc[next_candle_index]

    # Adjust pivot points if the next candle sets a new extreme
    if next_candle['Low'] < pivot_low:
        pivot_low = next_candle['Low']
        pivot_low_candle_index = next_candle_index
        # print(f"New pivot low found: {pivot_low} at index {pivot_low_candle_index}")
    else:
        move_in_low = next_candle['Low'] - pivot_low
        percent_low_move_threshold = pivot_low * (percent_threshold / 100)
        low_move_abs_threshold = max(absolute_threshold, percent_low_move_threshold)
        # Determine if the next extreme candle is a low or high point
        if move_in_low >= low_move_abs_threshold:
            if pivot_low_candle_index > start_candle_index:
                # print(f"Pivot point is the next extreme low point and pivot_low_candle_index is : {pivot_low_candle_index}")
                return (pivot_low_candle_index, pivot_low, "LOW")
            else:
                # print(f"Significant movement upwards from start_index identified at the candle index : {next_candle_index}..  exploring the high point now...")
                return next_extreme_high_point(df, start_candle_index=start_candle_index, input_candle_index=next_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold, absolute_threshold=absolute_threshold, pivot_high_candle_index=-1)

    if next_candle['High'] > pivot_high:
        pivot_high = next_candle['High']
        pivot_high_candle_index = next_candle_index
        # print(f"New pivot high found: {pivot_high} at index {pivot_high_candle_index}")
    else:
        move_in_high = pivot_high - next_candle['High']
        percent_high_move_threshold = pivot_high * (percent_threshold / 100)
        high_move_abs_threshold = max(absolute_threshold, percent_high_move_threshold)
        if move_in_high >= high_move_abs_threshold:
            if pivot_high_candle_index > start_candle_index:
                # print(f"New pivot high found: {pivot_high} at index {pivot_high_candle_index}")
                return (pivot_high_candle_index, pivot_high, "HIGH")
            else:
                # print(f"Significant movement downward from start_index identified at the candle index : {next_candle_index}..  exploring the low point now..")
                return next_extreme_low_point(df, start_candle_index=start_candle_index, input_candle_index=next_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold, absolute_threshold=absolute_threshold, pivot_low_candle_index=-1)

    # Continue searching if neither threshold is met
    # print("No significant move detected. Continuing search...")
    return next_extreme_candle(df, start_candle_index, next_candle_index, percent_threshold, move_relative_percent_threshold, absolute_threshold, pivot_high_candle_index, pivot_low_candle_index)



In [ ]:
# Make sure df_last_3_months['Date'] is in datetime format
df_last_3_months['Date'] = pd.to_datetime(df_last_3_months['Date'])

# Find the index for December 21, 2023
start_date = '2024-02-06'
start_candle_index = get_index(start_date,  df_last_3_months)

# Assuming next_extreme_high_point and get_index functions are already defined
next_extreme_index, next_extreme, extreme_type = next_extreme_candle(df_last_3_months, start_candle_index=start_candle_index, percent_threshold=0, move_relative_percent_threshold=0, absolute_threshold=0)
print(f"Next extreme {extreme_type} point > {next_extreme} is at date:", get_candle_date(df_last_3_months, next_extreme_index))



Next extreme HIGH point > 440.8999938964844 is at date: 2024-02-06 00:00:00


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1158667665.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Step 3: ZigZag**

In [ ]:
def zig_zag(df, percent_threshold=0,  move_relative_percent_threshold=0, absolute_threshold=0):
    """
    Generates a Zig Zag pattern by identifying a series of high and low extreme points
    in a given OHLC DataFrame.

    Parameters:
    - df: DataFrame containing OHLC data.
    - percent_threshold: The percentage movement threshold for considering an extreme point.
    - absolute_threshold: The absolute movement threshold for considering an extreme point.

    Returns:
    - A list of tuples, each containing the index of the extreme point and its type ("MAX" or "MIN").
    """

    # Ensure the 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Initialize the list to store the extreme points
    extreme_points = []

    # Start with the first candle in the DataFrame
    start_candle_index = df.index.min()
    current_type = None  # Keep track of the current extreme type to alternate between high and low

    while True:
        # Use next_extreme_candle to find the next extreme point from the current starting point
        next_point_index, next_extreme_value, extreme_type = next_extreme_candle(df, start_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold,  absolute_threshold=absolute_threshold)

        # Debugging statement to show the progress
        # print(f"Found extreme point at index {next_point_index} of type {extreme_type}")

        # Check if the next_point_index is None or if we've encountered a repeat
        if next_point_index == start_candle_index :
            # print("Encountered a repeat index")
            # Repeat index happens because that point itself is the extreme point hence it points at itsel
            if next_point_index is None:
                # print("Reached the End of dataframe")
                2==2
            break

        # Add the found extreme point to the list
        extreme_points.append((next_point_index, next_extreme_value, extreme_type))

        # Update start_candle_index for the next iteration
        start_candle_index = next_point_index

        # Check if we've reached the end of the DataFrame
        if start_candle_index == df.index.max():
            # print("Reached the end of DataFrame.")
            break

    return extreme_points


In [ ]:
# Assuming the DataFrame 'df_last_3_months' is already loaded and prepared
extreme_points = zig_zag(df_last_3_months, percent_threshold=5, move_relative_percent_threshold=0, absolute_threshold=0)

# for index, value, point_type in extreme_points:
#     print(f"Extreme point at index {index}, type: {point_type}, value: {value}, Date: {df_last_3_months.loc[index, 'Date']}")


New pivot high found: 137.6666717529297 at index 3996
Threshold met. Returning pivot high index: 3996
New pivot low found: 123.33333587646484 at index 4012
Threshold met. Returning pivot low index: 4012
New pivot high found: 136.3333282470703 at index 4023
New pivot high found: 136.93333435058594 at index 4024
New pivot high found: 137.26666259765625 at index 4025
New pivot high found: 137.46665954589844 at index 4026
New pivot high found: 138.5 at index 4028
New pivot high found: 138.6666717529297 at index 4031
New pivot high found: 142.0 at index 4032
New pivot high found: 144.13333129882812 at index 4033
Threshold met. Returning pivot high index: 4033
New pivot low found: 130.13333129882812 at index 4048
New pivot low found: 126.1999969482422 at index 4049
Threshold met. Returning pivot low index: 4049
New pivot high found: 138.0 at index 4056
New pivot high found: 138.39999389648438 at index 4057
New pivot high found: 138.6666717529297 at index 4058
New pivot high found: 139.800003

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2338709250.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



New pivot high found: 179.86666870117188 at index 4240
New pivot high found: 182.1666717529297 at index 4241
New pivot high found: 183.3333282470703 at index 4262
New pivot high found: 184.3000030517578 at index 4264
New pivot high found: 186.6666717529297 at index 4265
New pivot high found: 187.5 at index 4267
New pivot high found: 187.86666870117188 at index 4268
New pivot high found: 189.76666259765625 at index 4269
New pivot high found: 190.13333129882807 at index 4270
New pivot high found: 191.6999969482422 at index 4271
New pivot high found: 192.26666259765625 at index 4272
New pivot high found: 192.8999938964844 at index 4273
New pivot high found: 194.6666717529297 at index 4274
New pivot high found: 199.3333282470703 at index 4275
Threshold met. Returning pivot high index: 4275
New pivot low found: 183.6999969482422 at index 4290
New pivot low found: 182.1000061035156 at index 4291
New pivot low found: 182.0 at index 4292
Threshold met. Returning pivot low index: 4292
New pivot

In [ ]:
extreme_points2 = zig_zag(df_last_3_months, percent_threshold=0, move_relative_percent_threshold=0, absolute_threshold=0)

Threshold met. Returning pivot high index: 3982
New pivot low found: 130.3333282470703 at index 3984
Threshold met. Returning pivot low index: 3984
New pivot high found: 136.6666717529297 at index 3986
New pivot high found: 137.3333282470703 at index 3987
Threshold met. Returning pivot high index: 3987
New pivot low found: 131.03334045410156 at index 3989
Threshold met. Returning pivot low index: 3989
New pivot high found: 136.1999969482422 at index 3991
Threshold met. Returning pivot high index: 3991
Threshold met. Returning pivot low index: 3993
New pivot high found: 137.6666717529297 at index 3996
Threshold met. Returning pivot high index: 3996
New pivot low found: 129.8000030517578 at index 4002
Threshold met. Returning pivot low index: 4002
New pivot high found: 133.3333282470703 at index 4004
New pivot high found: 134.56666564941406 at index 4005
Threshold met. Returning pivot high index: 4005
Threshold met. Returning pivot low index: 4007
New pivot high found: 133.10000610351562

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2338709250.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



New pivot low found: 224.5666656494141 at index 4504
Threshold met. Returning pivot low index: 4504
Threshold met. Returning pivot high index: 4506
New pivot low found: 225.3333282470703 at index 4508
New pivot low found: 224.6666717529297 at index 4509
New pivot low found: 223.5666656494141 at index 4510
New pivot low found: 221.3333282470703 at index 4511
Threshold met. Returning pivot low index: 4511
New pivot high found: 233.6666717529297 at index 4513
New pivot high found: 233.86666870117188 at index 4514
Threshold met. Returning pivot high index: 4514
New pivot low found: 225.76666259765625 at index 4516
New pivot low found: 222.23333740234372 at index 4517
Threshold met. Returning pivot low index: 4517
New pivot high found: 237.3999938964844 at index 4519
Threshold met. Returning pivot high index: 4519
New pivot low found: 227.0 at index 4521
New pivot low found: 225.6666717529297 at index 4522
Threshold met. Returning pivot low index: 4522
Threshold met. Returning pivot high in

In [ ]:
df_last_3_months.index.min()

3981

In [ ]:
get_index("2023-02-24", df_last_3_months )

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



6827

In [ ]:
extreme_points2

[(3982, 135.23333740234375, 'HIGH'),
 (3984, 130.3333282470703, 'LOW'),
 (3987, 137.3333282470703, 'HIGH'),
 (3989, 131.03334045410156, 'LOW'),
 (3991, 136.1999969482422, 'HIGH'),
 (3993, 129.8333282470703, 'LOW'),
 (3996, 137.6666717529297, 'HIGH'),
 (4002, 129.8000030517578, 'LOW'),
 (4005, 134.56666564941406, 'HIGH'),
 (4007, 129.89999389648438, 'LOW'),
 (4009, 133.10000610351562, 'HIGH'),
 (4012, 123.33333587646484, 'LOW'),
 (4015, 135.3000030517578, 'HIGH'),
 (4018, 125.73332977294922, 'LOW'),
 (4026, 137.46665954589844, 'HIGH'),
 (4027, 135.39999389648438, 'LOW'),
 (4028, 138.5, 'HIGH'),
 (4030, 135.56666564941406, 'LOW'),
 (4033, 144.13333129882812, 'HIGH'),
 (4036, 138.3333282470703, 'LOW'),
 (4042, 143.53334045410156, 'HIGH'),
 (4049, 126.1999969482422, 'LOW'),
 (4058, 138.6666717529297, 'HIGH'),
 (4060, 129.8000030517578, 'LOW'),
 (4065, 135.56666564941406, 'HIGH'),
 (4066, 128.1999969482422, 'LOW'),
 (4072, 137.3333282470703, 'HIGH'),
 (4079, 131.73333740234375, 'LOW'),
 (40

**Step 4: ZigZag Plot**

In [ ]:
import plotly.graph_objects as go

# Assuming df_last_3_months is your DataFrame with the last 3 months of OHLC data
# Assuming extreme_points is the output from zig_zag(df_last_3_months), now including extreme values

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df_last_3_months['Date'],
                                     open=df_last_3_months['Open'],
                                     high=df_last_3_months['High'],
                                     low=df_last_3_months['Low'],
                                     close=df_last_3_months['Close'],
                                     name='Candlestick')])

# # Loop through the extreme_points to plot the high and low points using their exact values
# for index, extreme_value, extreme_type in extreme_points:
#     fig.add_trace(go.Scatter(x=[df_last_3_months.loc[index, 'Date']],
#                              y=[extreme_value],
#                              mode='markers',
#                              marker=dict(color='Green' if extreme_type == "LOW" else 'Red', size=8),
#                              name=f"{extreme_type} point"))

# # Loop through the extreme_points2 to plot the high and low points using their exact values
# for index, extreme_value, extreme_type in extreme_points2:
#     fig.add_trace(go.Scatter(x=[df_last_3_months.loc[index, 'Date']],
#                              y=[extreme_value],
#                              mode='markers',
#                              marker=dict(color='Green' if extreme_type == "LOW" else 'Red', size=8),
#                              name=f"{extreme_type} point"))

# For the trendline, use the dates and extreme values directly from the extreme_points
pivot_dates = [df_last_3_months.loc[index, 'Date'] for index, _, _ in extreme_points]
pivot_values = [extreme_value for _, extreme_value, _ in extreme_points]

# For the trendline, use the dates and extreme values directly from the extreme_points
pivot_dates2 = [df_last_3_months.loc[index, 'Date'] for index, _, _ in extreme_points2]
pivot_values2 = [extreme_value for _, extreme_value, _ in extreme_points2]

# Add a line connecting the pivot points to represent the zigzag trend
fig.add_trace(go.Scatter(x=pivot_dates,
                         y=pivot_values,
                         mode='lines',  # This will connect the points with lines
                         line=dict(color='Yellow', width=2),
                         name='Zigzag Trend'))

# Add a line connecting the pivot points to represent the zigzag trend
fig.add_trace(go.Scatter(x=pivot_dates2,
                         y=pivot_values2,
                         mode='lines',  # This will connect the points with lines
                         line=dict(color='Red', width=2),
                         name='Zigzag Trend'))

# Customize the layout
fig.update_layout(title='Stock Price Data with Zigzag Trendlines',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show the plot
fig.show()


# **Breather Identification**
---

**Step 1 : Uptrend_BREATHER_IDENTIFICATION**

In [ ]:
def Uptrend_BREATHER_IDENTIFICATION(start_point, Extreme_points, length_of_previous_trend_move, uptrend_correction_threshold=0, uptrend_retest_boundary_threshold=0):

    """
    Assess if the correction next to start_point is a small breather and can be ignored.
    """
    # print(f"Starting Uptrend_BREATHER_IDENTIFICATION with start_point index: {start_point}")

    if Extreme_points[start_point][2] == "LOW":
        # print("Starting point is a LOW, which is not suitable for uptrend analysis. High Point needed.")
        return ("High Point needed", 0)

    if start_point == 0:
        # print("Start point is the first point in the series. No previous point to compute move length.")
        return ("No previous point to compute move length", 0)

    if start_point >= len(Extreme_points) - 1:
        # print("Start point is the last point in the series. No points to assess after start_point.")
        return ("Start Point is the End Point", 0)

    start_point_value = Extreme_points[start_point][1]

    # Calculate upper and lower limits for the breather identification
    upper_limit = (1 + uptrend_retest_boundary_threshold / 100) * start_point_value
    # Lower Limit based on percentage correction
    lower_limit = start_point_value - (uptrend_correction_threshold / 100) * start_point_value
    # Lower_limit based on length of trend move
    # lower_limit = start_point_value - (uptrend_correction_threshold / 100) * length_of_previous_trend_move

    # print(f"Upper limit: {upper_limit}, Lower limit: {lower_limit}, based on start_point_value: {start_point_value} and prev_point_value: {prev_point_value}")

    no_of_skips = 0

    for i in range(start_point + 1, len(Extreme_points)):
        next_point_value = Extreme_points[i][1]
        # print(f"Evaluating point at index {i} with value {next_point_value}")

        if next_point_value > upper_limit:
            # print(f"Point at index {i} exceeds the upper limit. Trend Continued.")
            return ("Trend Continued", no_of_skips)
        elif next_point_value < lower_limit:
            # print(f"Point at index {i} falls below the lower limit. Trend Reversed.")
            return ("Trend Reversed", no_of_skips)
        else:
            no_of_skips += 1
            # print(f"Point at index {i} is within breather thresholds. Continuing analysis...")

    # print("Reached the end of Extreme_points without determining trend continuation or reversal.")
    return ("Breather Continued without trend reversal or continuation", 0)


In [ ]:
get_index('2023-07-24',df_last_3_months)

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



6926

In [ ]:
# Example date from which you want to start the assessment
given_date = '2023-07-24'

df_index = get_index(given_date, df_last_3_months)

# Find the start point in Extreme_points based on the given date
start_point = None
for i, (index, value, point_type) in enumerate(extreme_points):
    if index == df_index:
        start_point = i
        break

if start_point is None:
    print("No start point found for the given date within the extreme points.")
else:
    length_of_previous_trend_move = extreme_points[start_point][1] - extreme_points[start_point - 1][1]
    uptrend_correction_threshold = 40  # Example threshold value
    uptrend_retest_boundary_threshold = 0  # Example threshold value

    trend_status, no_of_skips = Uptrend_BREATHER_IDENTIFICATION(start_point, extreme_points, length_of_previous_trend_move=length_of_previous_trend_move,  uptrend_correction_threshold= uptrend_correction_threshold, uptrend_retest_boundary_threshold = uptrend_retest_boundary_threshold)

    print(f"Trend status: {trend_status}, Number of skips: {no_of_skips}")

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Trend status: Breather Continued without trend reversal or continuation, Number of skips: 0


**Step 2 : Downtrend_BREATHER_IDENTIFICATION**

In [ ]:
def Downtrend_BREATHER_IDENTIFICATION(start_point, Extreme_points, length_of_previous_trend_move, downtrend_correction_threshold=0, downtrend_retest_boundary_threshold=0):
    """
    Assess if the correction next to start_point in a downtrend is a small breather and can be ignored.

    Parameters:
    - start_point: The index of the point within the Extreme_points list.
    - Extreme_points: List of tuples containing (index, value, type) of extreme points.
    - length_of_previous_trend_move:  length of the move preceding the start_point
    - df: DataFrame containing the OHLC data.
    - downtrend_correction_threshold: Percentage threshold above which a correction is considered significant.
    - downtrend_retest_boundary_threshold: Percentage threshold below which the downtrend is considered continued.

    Returns:
    - A tuple containing trend_status ("Continued" / "Reversed" / None / "incomplete" / "End Point") and no_of_skips.
    """
    # print(f"Starting Down_BREATHER_IDENTIFICATION with start_point index: {start_point}")

    if start_point == 0 or start_point >= len(Extreme_points) - 1:
        if start_point == 0:
            # print("No previous point to compute move length.")
            return ("No previous point to compute move length", 0)
        else:
            # print("Start point is the last point in the series. No points to assess after start_point.")
            return ("Start Point is the End Point", 0)

    if Extreme_points[start_point][2] != "LOW":
        # print("Starting point is not a LOW, which is necessary for downtrend analysis. LOW Point needed.")
        return ("LOW Point needed", 0)

    start_point_low = Extreme_points[start_point][1]

    lower_limit = (1 - downtrend_retest_boundary_threshold / 100) * start_point_low
    # Upper_limit based on percentage of the start point
    upper_limit = start_point_low + (downtrend_correction_threshold / 100) * start_point_low

    # Upper_limit based on length of trend move
    # upper_limit = start_point_low + (downtrend_correction_threshold / 100) * length_of_previous_trend_move

    # print(f"Calculated lower_limit: {lower_limit}, upper_limit: {upper_limit} based on start_point_low: {start_point_low} and length_of_trend_move: {length_of_trend_move}")

    no_of_skips = 0

    for i in range(start_point + 1, len(Extreme_points)):
        next_point_value = Extreme_points[i][1]
        # print(f"Evaluating point at index {i} with value {next_point_value}")

        if next_point_value < lower_limit:
            # print(f"Point at index {i} falls below the lower limit. Trend Continued.")
            return ("Trend Continued", no_of_skips)
        elif next_point_value > upper_limit:
            # print(f"Point at index {i} exceeds the upper limit. Trend Reversed.")
            return ("Trend Reversed", no_of_skips)
        else:
            no_of_skips += 1
            # print(f"Point at index {i} is within the breather thresholds. Continuing analysis...")

    # print("Reached the end of Extreme_points without determining trend continuation or reversal.")
    return ("Breather Continued without trend reversal or continuation", 0)


In [ ]:
# Example date from which you want to start the assessment
given_date = '2024-01-10'

df_index = get_index(given_date, df_last_3_months)

# Find the start point in Extreme_points based on the given date
start_point = None
for i, (index, value, point_type) in enumerate(extreme_points):
    if index == df_index:
        start_point = i
        break

if start_point is None:
    print("No start point found for the given date within the extreme points.")
else:
    length_of_previous_trend_move = extreme_points[start_point - 1][1] - extreme_points[start_point][1]
    downtrend_correction_threshold = 10  # Example threshold value
    downtrend_retest_boundary_threshold = 0  # Example threshold value

    trend_status, no_of_skips = Downtrend_BREATHER_IDENTIFICATION(start_point, extreme_points, length_of_previous_trend_move=length_of_previous_trend_move, downtrend_correction_threshold= downtrend_correction_threshold, downtrend_retest_boundary_threshold = downtrend_retest_boundary_threshold)

    print(f"Trend status: {trend_status}, Number of skips: {no_of_skips}")

No start point found for the given date within the extreme points.


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Step 2: BREATHER_FILTER**

In [ ]:
def BREATHER_FILTER(Extreme_points, uptrend_correction_threshold, downtrend_correction_threshold, uptrend_retest_boundary_threshold, downtrend_retest_boundary_threshold):
    """
    Filters out small corrections in large trends from a list of extreme points.

    Parameters:
    - Extreme_points: List of tuples containing (index, value, type) of extreme points.
    - uptrend_correction_threshold: Percentage threshold for uptrend corrections.
    - downtrend_correction_threshold: Percentage threshold for downtrend corrections.
    - uptrend_retest_boundary_threshold: Percentage threshold for retesting in uptrends.
    - downtrend_retest_boundary_threshold: Percentage threshold for retesting in downtrends.

    Returns:
    - A filtered list of extreme points.
    """
    # Validation check to ensure Extreme_points is not empty
    if not Extreme_points:
        print("Extreme_points list is empty. No points available for filtering.")
        return []

    filtered_extreme_points = [Extreme_points[0]]  # Initialize with the first point

    i = 1  # Start from the second point
    while i < len(Extreme_points):
        current_point = Extreme_points[i]
        # print(f"Evaluating point at index {i}: {current_point}")

        if current_point[2] == "HIGH":
            length_of_previous_trend_move = current_point[1] - filtered_extreme_points[-1][1]
            trend_status, no_of_skips = Uptrend_BREATHER_IDENTIFICATION(i, Extreme_points, length_of_previous_trend_move, uptrend_correction_threshold, uptrend_retest_boundary_threshold)
        elif current_point[2] == "LOW":
            length_of_previous_trend_move = filtered_extreme_points[-1][1]  -  current_point[1]
            trend_status, no_of_skips = Downtrend_BREATHER_IDENTIFICATION(i, Extreme_points, length_of_previous_trend_move, downtrend_correction_threshold, downtrend_retest_boundary_threshold)

        # print(f"Trend status: {trend_status}, No of skips: {no_of_skips}")

        if trend_status == "Start Point is the End Point":
            filtered_extreme_points.append(current_point)
            return filtered_extreme_points
        elif trend_status == "Trend Reversed":
            filtered_extreme_points.append(current_point)
            i += no_of_skips + 1  # Move to the next extreme point for evaluation
        elif trend_status == "Trend Continued":
            i += no_of_skips + 1  # Skip minor correction points
        elif trend_status == "Breather Continued without trend reversal or continuation":
            # Include all points starting from current_point till the end
            filtered_extreme_points.extend(Extreme_points[i:])
            break

    return filtered_extreme_points




In [ ]:
# Example usage:
# Assuming Extreme_points and thresholds are defined
uptrend_correction_threshold = 9.99
downtrend_correction_threshold = 9.99
filtered_extreme_points = BREATHER_FILTER(extreme_points, uptrend_correction_threshold=uptrend_correction_threshold,downtrend_correction_threshold=downtrend_correction_threshold, uptrend_retest_boundary_threshold=0.1, downtrend_retest_boundary_threshold=0.1)


**Plot the output**
---

In [ ]:
import plotly.graph_objects as go

# Assuming df_last_3_months is your DataFrame with the last 3 months of OHLC data
# Assuming Extreme_points and filtered_extreme_points are already defined

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df_last_3_months['Date'],
                                     open=df_last_3_months['Open'],
                                     high=df_last_3_months['High'],
                                     low=df_last_3_months['Low'],
                                     close=df_last_3_months['Close'],
                                     name='Candlestick')])

# Function to extract date and value for plotting
def extract_date_value(points, df):
    # Corrected function to ensure 'idx' is defined in both comprehensions
    dates = [df.loc[idx, 'Date'] for idx, value, point_type in points if idx in df.index]
    values = [value for idx, value, point_type in points if idx in df.index]
    return dates, values

# Original Extreme Points (with trend lines)
dates, values = extract_date_value(extreme_points, df_last_3_months)
fig.add_trace(go.Scatter(x=dates, y=values, mode='lines+markers',
                         line=dict(color='Blue', width=2),
                         marker=dict(color='LightSkyBlue', size=1),
                         name='Original Trend'))

# Filtered Extreme Points (with trend lines)
filtered_dates, filtered_values = extract_date_value(filtered_extreme_points, df_last_3_months)
fig.add_trace(go.Scatter(x=filtered_dates, y=filtered_values, mode='lines+markers',
                         line=dict(color='DarkGreen', width=2),
                         marker=dict(color='Green', size=1),
                         name='Filtered Trend'))

# Customize the layout
fig.update_layout(title='Stock Price Data with Trends',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Show the plot
fig.show()


In [ ]:
start_index = get_index('2023-11-01', df_last_3_months)
end_index = get_index('2024-01-04', df_last_3_months)

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# **Stage2 Identification**
---

**price Volume Trend Check**

In [ ]:
import numpy as np

def priceVolumeTrendCheck(df, start_index, end_index, upDownHighVolumeRatioThreshold=1.25, upTickPercentile=0.75, DownTickPercentile=0.25, change_type='CLOSE'):
    """
    Assess the ratio of significant upticks to downticks with high volume in a given period within a stock DataFrame.

    Parameters:
    - df: DataFrame containing stock OHLC data and Volume.
    - start_index: Index in df from which the analysis period starts.
    - end_index: Index in df at which the analysis period ends.
    - upDownHighVolumeRatioThreshold: Threshold ratio of significant upticks to downticks with high volume.
    - upTickPercentile: Percentile threshold for considering an uptick significant.
    - DownTickPercentile: Percentile threshold for considering a downtick significant.
    - change_type: Method to calculate price change ('HIGH-LOW', 'CLOSE', 'OPEN-CLOSE').

    Returns:
    - Ratio of significant upticks with high volume to significant downticks with high volume.
    """

    # Validate that provided indices are within the DataFrame's bounds and in correct order
    if start_index not in df.index or end_index not in df.index or start_index > end_index:
        # print("Invalid start or end index. Please ensure indices are within DataFrame bounds and start_index <= end_index.")
        return None

    # Calculate the 10-day rolling average of volume for each day in the dataset
    # Ensures that the rolling average for the initial days can include data preceding the start_index
    df['Vol_10D_avg'] = df['Volume'].rolling(window=10, min_periods=1).mean()

    # Calculate price changes within the entire DataFrame based on the specified change_type
    if change_type == 'CLOSE':
        df['PriceChange'] = df['Close'].diff()  # Daily close price change
    elif change_type == 'OPEN-CLOSE':
        df['PriceChange'] = df['Close'] - df['Open']  # Intraday price change
    else:
        # print("Invalid change_type specified.")
        return None

    # Filter the DataFrame for the analysis period
    df_filtered = df.loc[start_index:end_index]

    # Initialize counters for significant upticks and downticks with high volume
    count_up_tick_high_vol = 0
    count_down_tick_high_vol = 0

    # Setting upTickThreshold
    upTickThreshold = df['PriceChange'].quantile(upTickPercentile)
    downTickThreshold = df['PriceChange'].quantile(DownTickPercentile)

    # Iterate over the filtered DataFrame to count significant upticks and downticks with high volume
    for idx, row in df_filtered.iterrows():
        # Check if the current day's volume is above its 10-day average
        if row['Volume'] > row['Vol_10D_avg']:
            # Check for significant upticks
            if row['PriceChange'] >= upTickThreshold:
                count_up_tick_high_vol += 1
            # Check for significant downticks
            elif row['PriceChange'] <= downTickThreshold:
                count_down_tick_high_vol += 1

    # Calculate and return the ratio of significant upticks to downticks with high volume
    if count_down_tick_high_vol == 0:  # Handle division by zero
        ratio = 999  # Use a high value to indicate an overwhelmingly positive trend
    else:
        ratio = count_up_tick_high_vol / count_down_tick_high_vol
    # print(f"Count of total ticks : {end_index - start_index}")
    # print(f"Count of significant upticks with high volume: {count_up_tick_high_vol}")
    # print(f"Count of significant downticks with high volume: {count_down_tick_high_vol}")
    # print(f"Ratio of upticks to downticks with high volume: {ratio}")

    return ratio


In [ ]:
# Assuming df_last_3_months is loaded and get_index function is defined

# Obtain start and end indices for the analysis period
start_index = get_index('2023-11-01', df_last_3_months)
end_index = get_index('2024-01-04', df_last_3_months)
print(f"start_index : {start_index}")
print(f"end_index : {end_index}")

# Specify parameters for the analysis
upDownHighVolumeRatioThreshold = 1.25  # Threshold ratio of significant upticks to downticks with high volume
upTickPercentile = 0.75  # Percentile threshold for considering an uptick significant
DownTickPercentile = 0.25  # Percentile threshold for considering a downtick significant
change_type = 'CLOSE'  # Method to calculate price change

# Call priceVolumeTrendCheck function with the obtained indices and specified parameters
upDownHighVolumeRatio = priceVolumeTrendCheck(
    df_last_3_months,
    start_index,
    end_index,
    upDownHighVolumeRatioThreshold,
    upTickPercentile,
    DownTickPercentile,
    change_type
)

print("UpDownHighVolumeRatio:", upDownHighVolumeRatio)


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



start_index : 6994
end_index : 7037
UpDownHighVolumeRatio: 1.1428571428571428


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**moving Average Check**

In [ ]:
import pandas as pd
import numpy as np

def movingAverageCheck(start_index, end_index, df, initial_noise_period_proportion=15):
    """
    Checks if the moving averages are in the correct order and if the 200-day MA is increasing,
    accounting for an initial noise period.

    Parameters:
    - start_index: The index from which to start the check.
    - end_index: The index at which to end the check.
    - df: DataFrame containing stock data.
    - initial_noise_period_proportion: Proportion of the total move considered as initial noise.

    Returns:
    - True if checks pass, False otherwise.
    """

    # Debug log for moving averages
    # print("Added moving averages to DataFrame with min_periods=1.")

    # Check 1: 200-day MA should be increasing
    for i in range(start_index + 1, end_index + 1):  # Start from the next day of start_index
        if df.at[i, 'MA200'] <= df.at[i - 1, 'MA200']:
            print(f"Check 1 failed: 200-day MA not increasing at index {i}. start_index @{start_index} & end_index @{end_index}")
            return False

    # Determine the initial noise period index
    total_time = end_index - start_index
    initial_noise_period_index = int(start_index + np.ceil(initial_noise_period_proportion / 100.0 * total_time))

    # Debug log for initial noise period index
    print(f"Initial noise period index: {initial_noise_period_index}.")

    # Check 2: After initial noise period, 50-day MA > 150-day MA > 200-day MA
    for i in range(initial_noise_period_index, end_index + 1):
        if not (df.at[i, 'MA50'] > df.at[i, 'MA150'] > df.at[i, 'MA200']):
            print(f"Check 2 failed: MA order incorrect at index {i}.")
            return False

    # If all checks pass
    return True


In [ ]:
# Fetch start and end indices based on dates
start_index = get_index('2023-11-01', df_last_3_months)
end_index = get_index('2024-01-04', df_last_3_months)

# Call movingAverageCheck function
result = movingAverageCheck(start_index, end_index, df_last_3_months, initial_noise_period_proportion=10)

# Print the result
print("Moving averages check result:", result)


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Initial noise period index: 6999.
Check 2 failed: MA order incorrect at index 7002.
Moving averages check result: False


**super up trend**

In [ ]:
def superUpTrend(start_index, end_index, df, min_jump=30, min_time=10, max_time=320, min_slope=0.1, comparison_type='HIGH-LOW', TimeLookBack=None, priceVolumeThreshold=1.25):
    """
    Determine if there has been an exceptional rise in stock prices between two points in time.

    Parameters:
    - start_index, end_index: Indices defining the analysis period within the DataFrame.
    - df: DataFrame containing stock OHLC data and Volume.
    - min_jump: Minimum percentage jump in price to qualify as a super uptrend.
    - min_time, max_time: Minimum and maximum time (in candles) for the trend.
    - min_slope: Minimum slope of the price jump to qualify as a super uptrend.
    - comparison_type: 'HIGH-LOW' or 'CLOSE', determines how price jump is calculated.
    - TimeLookBack: Number of candles to look back from end_index to ensure stock is in new territory.
    - priceVolumeThreshold: Minimum price-volume trend ratio to qualify as a super uptrend.

    Returns:
    - JSON object containing analysis results.
    """
    output_json = {
        "superUpTrend": True,
        "price_jump": None,
        "time_taken": None,
        "slope": None,
        "desc": ""
    }

    # Calculate price jump and check if the stock is in new territory
    if comparison_type == 'HIGH-LOW':
        price_jump = (df.at[end_index, 'High'] - df.at[start_index, 'Low']) / df.at[start_index, 'Low'] * 100
        # Define the lookback period based on TimeLookBack and available data
        lookBackStart = max(end_index - TimeLookBack, df.index.min()) if TimeLookBack else df.index.min()
        # Use DataFrame to determine the maximum value in the lookback range directly
        maxLookBackHigh = df.loc[lookBackStart:end_index, 'High'].max()
        stockInNewTerritoryFlag = maxLookBackHigh >= df.at[end_index, 'High']
        # print(f"price jump is {price_jump}")
        # print(f"Max High in LookBack Range: {maxLookBackHigh} .... end index High: {df.at[end_index, 'High']}")

    else:  # 'CLOSE'
        price_jump = (df.at[end_index, 'Close'] - df.at[start_index, 'Close']) / df.at[start_index, 'Close'] * 100
        # Define the lookback period based on TimeLookBack and available data
        lookBackStart = max(end_index - TimeLookBack, df.index.min()) if TimeLookBack else df.index.min()
        # Use DataFrame to determine the maximum value in the lookback range directly
        maxLookBackClose = df.loc[lookBackStart:end_index, 'Close'].max()
        stockInNewTerritoryFlag = maxLookBackClose >= df.at[end_index, 'Close']
        # print(f"Max Close in LookBack Range: {maxLookBackClose} .... end index Close: {df.at[end_index, 'Close']}")


    #  In pandas, df.at provides a very fast and efficient way to access a scalar value by its row label and column label.
    #  This method is optimized for accessing a single value: if you know the row label and the column name, df.at is the recommended method to get or set the value at that position.

    # Calculate time taken and slope
    time_taken = end_index - start_index
    if time_taken > 0:
      slope = price_jump / time_taken
    else:
        slope = float('inf')
        print(f"end index is {end_index} and start index is {start_index}")
        print("SLOPE IS INFINITE..  SOME ERROR!!")

    # Check all conditions
    if price_jump < min_jump:
        output_json["superUpTrend"] = False
        output_json["desc"] += f"Price jump < {min_jump}%. "

    if (time_taken < min_time) or (max_time is not None and time_taken >= max_time):
        output_json["superUpTrend"] = False
        output_json["desc"] += f"Time taken {time_taken} not in range {min_time} & {max_time}. "

    if slope < min_slope:
        output_json["superUpTrend"] = False
        output_json["desc"] += f"Slope < {min_slope}. "

    if not stockInNewTerritoryFlag:
        output_json["superUpTrend"] = False
        output_json["desc"] += f"Stock is not in new territory in the last {TimeLookBack} candles. "

    if not movingAverageCheck(start_index, end_index, df, initial_noise_period_proportion = 15):
        output_json["superUpTrend"] = False
        output_json["desc"] += f"Moving average check is FAILED and time_taken is {time_taken}"

    # Call priceVolumeTrendCheck function with the obtained indices and specified parameters
    upDownHighVolumeRatio = priceVolumeTrendCheck(
        df, start_index, end_index, priceVolumeThreshold, 0.75, 0.25, 'CLOSE'
    )

    if upDownHighVolumeRatio < priceVolumeThreshold:
        output_json["superUpTrend"] = False
        output_json["desc"] += f"High volume on rise and low volume on decline not evidenced; upDownHighVolumeRatio is {upDownHighVolumeRatio} < {priceVolumeThreshold}. "

    # Calculate max decline over 5-day periods directly
    df['5Day_Change'] = df['Low'] - df['High'].shift(-4)
    df['5Day_Pct_Change'] = df['5Day_Change'] / df['High'].shift(-4)*100
    filtered_df = df.loc[start_index : end_index]
    max_5day_downward_pct_change = filtered_df['5Day_Pct_Change'].min()

    # Calculate max decline over 1-day period directly
    df['1Day_Change'] = df['Low'] - df['High'].shift(-1)
    df['1Day_Pct_Change'] = df['1Day_Change'] / df['High'].shift(-1)*100
    filtered_df = df.loc[start_index : end_index]
    max_1day_downward_pct_change = filtered_df['1Day_Pct_Change'].min()

    # Update output_json with calculated values
    output_json.update({
        "price_jump": price_jump,
        "time_taken": time_taken,
        "slope": slope,
        "max_1D_decline": max_1day_downward_pct_change,
        "max_5D_decline": max_5day_downward_pct_change
    })
    print(f"output_json : {output_json}")

    return output_json

In [ ]:
# Assuming df_last_3_months is your DataFrame and superUpTrend is defined as per previous instructions

# Obtain start and end indices for the analysis period
start_index = get_index('2023-11-01', df_last_3_months)
end_index = get_index('2024-01-04', df_last_3_months)

print(f"start_index : {start_index}")
print(f"end_index : {end_index}")

# Call the superUpTrend function with the obtained indices and specified parameters
result = superUpTrend(
    start_index=start_index,
    end_index=end_index,
    df=df_last_3_months,
    min_jump=10,
    min_time=10,
    max_time=None,
    min_slope=0.1,
    comparison_type='HIGH-LOW',
    TimeLookBack=60,  # Example: look back 60 candles from end_index to check if stock is in new territory
    priceVolumeThreshold=1.25
)

print(result)


start_index : 6994
end_index : 7037
Initial noise period index: 7001.
Check 2 failed: MA order incorrect at index 7002.
output_json : {'superUpTrend': False, 'price_jump': 13.149239061581977, 'time_taken': 43, 'slope': 0.3057962572460925, 'desc': 'Moving average check is FAILED and time_taken is 43High volume on rise and low volume on decline not evidenced; upDownHighVolumeRatio is 1.1428571428571428 < 1.25. ', 'max_1D_decline': -4.163014954883932, 'max_5D_decline': -6.458557588805166}
{'superUpTrend': False, 'price_jump': 13.149239061581977, 'time_taken': 43, 'slope': 0.3057962572460925, 'desc': 'Moving average check is FAILED and time_taken is 43High volume on rise and low volume on decline not evidenced; upDownHighVolumeRatio is 1.1428571428571428 < 1.25. ', 'max_1D_decline': -4.163014954883932, 'max_5D_decline': -6.458557588805166}


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

**NextLowPoint**

In [ ]:
def NextLowPoint(start_point, extreme_points, price_jump, max_time_taken = 250):
    """
    Find the next significant low point and the subsequent high point that meets the price jump criterion.

    Parameters:
    - low_point_index: Index within extreme_points where the search begins.
    - extreme_points: Array of tuples containing extreme points (index, value, type).
    - price_jump: The minimum jump in percentage required to identify a probable high point.

    Returns:
    - Tuple of (NextLowPoint, ProbableHighPoint) where ProbableHighPoint could be None if not found.
    """

    current_low_point = start_point
    current_low_point_index = extreme_points[current_low_point][0]
    current_low_point_value = extreme_points[current_low_point][1]
    NextHighPoint = None

    # Outer loop is to iterate through the low_point if the first low_point does not give any result
    while True:
        # Inner loop is to iterate through the next high points given a fixed low point
        for i in range(current_low_point + 1, len(extreme_points)):
            evaluated_point = extreme_points[i]
            evaluated_point_index, evaluated_point_value, evaluated_point_type = extreme_points[i][0], extreme_points[i][1], extreme_points[i][2]
            evaluated_point = i

            if evaluated_point_type == "LOW":
                if evaluated_point_value <= current_low_point_value:
                    current_low_point = evaluated_point
                    current_low_point_index = evaluated_point_index
                    current_low_point_value = evaluated_point_value
                    NextHighPoint = None  # Reset because a new lower low point is found

            elif evaluated_point_type == "HIGH":
                print(f"supertrend called from low_point {current_low_point_index} and high_point is {evaluated_point_index}")
                result = superUpTrend(
                    start_index=current_low_point_index,
                    end_index=evaluated_point_index,
                    df=df,
                    min_jump=price_jump,
                    min_time=10,
                    max_time=None,
                    min_slope=0.1,
                    comparison_type='HIGH-LOW',
                    TimeLookBack=500,  # Example: look back 60 candles from end_index to check if stock is in new territory
                    priceVolumeThreshold=1.25
                )
                if result['superUpTrend']:
                    NextHighPointIndex = evaluated_point_index
                    NextHighPoint = evaluated_point
                    break  # Exit loop as the target high point is found
                else:
                    if result["time_taken"] > max_time_taken:
                        print("Exceeded the max_time_taken hence giving up the next_high_point search in NextLowPoint function")
                        max_time_exceeded = True
                        break

        if NextHighPoint is not None:
            # print(f"Next high point identified @ index {NextHighPoint} from current low point @ index {current_low_point}")
            # break the while loop
            break

        else:
            print(f"No good jump identified from current low point @ index {current_low_point}..  moving to next low point")
            if current_low_point + 2 < len(extreme_points):
                current_low_point = current_low_point + 2
                current_low_point_index = extreme_points[current_low_point][0]
                current_low_point_value = extreme_points[current_low_point][1]
            else:
                current_low_point = None
                break

    return (current_low_point, NextHighPoint)




In [ ]:
start_point = 2  # Starting from the first low point in the given example
price_jump = 30  # Looking for a 5% price jump
print(f"low_point_index :: {start_point}")

next_low_point, probable_high_point = NextLowPoint(start_point, extreme_points, price_jump)
print(f"next_low_point >> {next_low_point}... probable_high_point>> {probable_high_point}")
if next_low_point is not None:
    low_point_index = extreme_points[next_low_point][0]
if probable_high_point is not None:
    probable_high_point_index = extreme_points[probable_high_point][0]
else:
    probable_high_point_index = None
print(f"Next Low Point Index: {next_low_point}, Next High Point Index: {probable_high_point}")
if next_low_point is not None:
    print(f"start_date is {get_candle_date(df_last_3_months, low_point_index)}.. end_date is {get_candle_date(df_last_3_months, probable_high_point_index)}")


low_point_index :: 2
supertrend called from low_point 4049 and high_point is 4155
Initial noise period index: 4065.
Check 2 failed: MA order incorrect at index 4095.
output_json : {'superUpTrend': False, 'price_jump': 33.57105366535587, 'time_taken': 106, 'slope': 0.31670805344675346, 'desc': 'Moving average check is FAILED and time_taken is 106', 'max_1D_decline': -7.644439697265624, 'max_5D_decline': -8.525251908735795}
supertrend called from low_point 4049 and high_point is 4163
Initial noise period index: 4067.
Check 2 failed: MA order incorrect at index 4095.
output_json : {'superUpTrend': False, 'price_jump': 28.57897505671565, 'time_taken': 114, 'slope': 0.25069276365540044, 'desc': 'Price jump < 30%. Moving average check is FAILED and time_taken is 114', 'max_1D_decline': -7.644439697265624, 'max_5D_decline': -8.525251908735795}
supertrend called from low_point 4049 and high_point is 4275
Initial noise period index: 4083.
Check 2 failed: MA order incorrect at index 4095.
output

**NextHighPoint**

In [ ]:
def NextHighPoint(low_point, Next_High_Point, extreme_points, df, price_jump, max_time_taken = 250, stage2_potential_df=None, correction_factor = 40, stock_name = None):
    """
    Identify the next high point for a given low point and confirm if it's a super uptrend.
    """

    current_high_point = Next_High_Point
    # Retrieve the index for the low_point and Probable_High_Point within extreme_points
    low_point_index = extreme_points[low_point][0]

    current_high_point_index = extreme_points[current_high_point][0]
    final_high_point = None

    if Next_High_Point is None:
        Next_High_Point = low_point + 1

    if Next_High_Point <= low_point:
        print("Incorrect Probable_High_Point provided to NextHighPoint function..  it can't be lesser than low_point")
        return (None, None)

    while current_high_point + 1 < len(extreme_points):

        # Check for super uptrend between the current low and new high point
        # print("Trying to confirm super UpTrend")
        # print(f"start_index is {low_point_index}... and end_index is {current_high_point_index} ")
        result = superUpTrend(
            start_index=low_point_index,
            end_index=current_high_point_index,
            df=df,
            min_jump=price_jump,
            min_time=10,
            max_time=None,
            min_slope=0.1,
            comparison_type='HIGH-LOW',
            TimeLookBack=500,  # Example: look back 500 candles from end_index to check if stock is in new territory
            priceVolumeThreshold=1.25
        )
        max_time_exceeded = False
        if result["superUpTrend"]:
              print("Super uptrend confirmed.")
              # Apppend the record to new DataFrame
              # Create a new DataFrame for the row you want to append
              new_row_df = pd.DataFrame([{
                  'Stock_name': stock_name,  # This should be dynamic based on your data
                  'Start_Index': low_point_index,
                  'End_Index': current_high_point_index,
                  'Price_Percentage_Change': result["price_jump"],
                  'Slope': result["slope"],
                  'Change_Time_Taken': result["time_taken"],
                  'Start_date': get_candle_date(df,low_point_index ),  # This should be derived from your data
                  'End_Date': get_candle_date(df,current_high_point_index ),  # This should be derived from your data
                  'max_1D_decline': result["max_1D_decline"],
                  'max_5D_decline': result["max_5D_decline"],
                  'current_high_point': current_high_point
              }], columns=stage2_potential_df.columns)
              print(f"New record inserted {new_row_df}")
              stage2_potential_df = pd.concat([stage2_potential_df, new_row_df], ignore_index=True)
              final_high_point = current_high_point
        else:
              print(f"result[time_taken] is ", result["time_taken"])
              if result["time_taken"] > max_time_taken:
                  print("Exceeded the max_time_taken hence giving up the next_high_point search")
                  max_time_exceeded = True
                  break
        if max_time_exceeded:
            break

        found_higher_high = False
        found_low_correction = False
        found_significant_side_move = False

        for i in range(current_high_point + 1, len(extreme_points)):
            point_index, point_value, point_type = extreme_points[i]

            if point_type == "LOW" :
                # Ensure that there has not been a significant decline.  If there is more than say 40% correction in upmove it can be significant decline
                if point_value < (extreme_points[current_high_point][1] - extreme_points[low_point][1]) * correction_factor /100 + extreme_points[low_point][1]:
                    found_higher_high = False
                    found_low_correction = True
                    # print(f"After a new high point @{extreme_points[current_high_point][0]}, correction happened>>>")
                if final_high_point is not None:
                    upMoveTimeTaken = extreme_points[current_high_point][0] - extreme_points[low_point][0]
                    sideWaysTimeTaken = extreme_points[i][0] - extreme_points[final_high_point][0]
                    # Ensure that there has not been a significant SIDE WAY MOVE.  If there is more time taken than total up_move then that's the case.
                    if sideWaysTimeTaken > upMoveTimeTaken:
                        found_significant_side_move = True

            if point_type == "HIGH" and point_value > extreme_points[current_high_point][1]:
                # print(f"Found new HIGH point at index {i} with value {point_value}")
                # Update current high point and continue the search
                current_high_point = i
                current_high_point_index = extreme_points[current_high_point][0]
                found_higher_high = True
                break

        if not found_higher_high:
            # print("No further 'HIGH' points found that exceed the current high point.")
            break

        if found_low_correction:
            print('Significant correction found hence skipping the next_high_point search instead new_low_point to be found')
            break

        if found_significant_side_move:
            print('Significant side way move found hence skipping the next_high_point search, instead new_low_point to be found')
            break


    return final_high_point, stage2_potential_df


In [ ]:

# Placeholder setup for demonstration
low_point = 0  # Assuming this is the index of the low point in your extreme_points
Probable_High_Point = 3  # Assuming this is the index of a high point in your extreme_points
price_jump = 10  # Example price jump percentage criterion

low_point_index = get_index('2023-11-01',df_last_3_months)
low_point_index = 6092
for position, (index, _, _) in enumerate(extreme_points):
    if index == low_point_index:
        low_point =  position
print(f"low_point_index = {low_point_index}")
print(f"low_point = {low_point}")

Probable_High_Point = low_point + 1

# Initialize stage2_potential_df if not already present
stage2_potential_df = pd.DataFrame(columns=['Stock_name', 'Start_Index', 'End_Index',
                                                'Price_Percentage_Change', 'Slope', 'Change_Time_Taken',
                                                'Start_date', 'End_Date', 'max_1D_decline', 'max_5D_decline', 'current_high_point' ])

# Call the NextHighPoint function
current_high_point_index, updated_stage2_potential_df = NextHighPoint(
    low_point,
    Probable_High_Point,
    extreme_points,
    df=df_last_3_months,
    price_jump = 1,
    stage2_potential_df = stage2_potential_df,
    correction_factor = 40,
    stock_name='ITC'
)

# Example output
print("Updated stage2_potential_df:")
print(updated_stage2_potential_df)

# Note: This example assumes the NextHighPoint function, and the superUpTrend function have been correctly
# defined and integrated as per earlier discussions. The superUpTrend function needs to be compatible
# with the parameters passed from NextHighPoint and should return a JSON object with super uptrend analysis results.


low_point_index = 6092
low_point = 77
Check 1 failed: 200-day MA not increasing at index 6093. start_index @6092 & end_index @6097
output_json : {'superUpTrend': False, 'price_jump': 31.12926597065733, 'time_taken': 5, 'slope': 6.225853194131465, 'desc': 'Time taken 5 not in range 10 & None. Moving average check is FAILED and time_taken is 5', 'max_1D_decline': -17.988668555240793, 'max_5D_decline': -17.70538243626062}
result[time_taken] is  5
Significant correction found hence skipping the next_high_point search instead new_low_point to be found
Updated stage2_potential_df:
Empty DataFrame
Columns: [Stock_name, Start_Index, End_Index, Price_Percentage_Change, Slope, Change_Time_Taken, Start_date, End_Date, max_1D_decline, max_5D_decline, current_high_point]
Index: []


C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

**Stage2 Identifier**

In [ ]:
import pandas as pd

def Stage2Identifier(stock_name, extreme_points, price_jump, df):
    """
    Identifies potential VCP stage 2 moves within the entire stock data series.

    Parameters:
    - stock_name: Name of the stock being analyzed.
    - extreme_points: Array of tuples containing extreme points (index, value, type).
    - price_jump: The minimum price jump percentage required to consider a move significant.
    - df: DataFrame containing OHLC data for the stock.

    Returns:
    - DataFrame containing identified stage 2 potential moves.
    """
    # Initialize the DataFrame to store potential stage 2 moves
    stage2_potential_df = pd.DataFrame(columns=['Stock_name', 'Start_Index', 'End_Index',
                                                'Price_Percentage_Change', 'Slope', 'Change_Time_Taken',
                                                'Start_date', 'End_Date', 'max_1D_decline', 'max_5D_decline', 'current_high_point'])
    start_point = 0

    print(f"Starting Stage2Identifier for stock: {stock_name} with price jump criterion: {price_jump}%")

    while start_point < len(extreme_points) - 1:
        print(f"Analyzing from start point: {start_point} in extreme points.")
        current_low_point, Next_High_Point = NextLowPoint(start_point, extreme_points, price_jump)
        if current_low_point is not None and Next_High_Point is not None:
            print(f"current_low_point index is {extreme_points[current_low_point][0]} and date is  {get_candle_date(df, extreme_points[current_low_point][0])} ")

        if Next_High_Point is None and current_low_point is None and start_point < len(extreme_points) - 1:
            print(f"start_point is {start_point} and extreme_points length is {len(extreme_points)}")
            print("No low point to high point left to be evaluated. Ending analysis.")
            break
        else:
            print(f"Next high point found at {extreme_points[Next_High_Point][0]}. LOW Point at {extreme_points[current_low_point][0]} Attempting to identify a stage 2 move.")
            current_high_point, stage2_potential_df = NextHighPoint(
                low_point=current_low_point,
                Next_High_Point=Next_High_Point,
                extreme_points=extreme_points,
                df=df,
                price_jump=price_jump,
                stage2_potential_df=stage2_potential_df,
                correction_factor = 40,
                stock_name=stock_name
            )


            if current_high_point is None:
                print("No new high point identified. Exiting the loop.")
                break
            else:
                print(f"New high point identified at {current_high_point}. Moving to the next analysis point.")
                # Assuming NextHighPoint returns the index in extreme_points, find the next start point
                start_point =  current_high_point + 1

    print("Stage 2 identification complete.")
    return stage2_potential_df



In [ ]:
# Call the Stage2PotentialIdentifier function for stock "ITC" with a price jump criterion of 10%
stage2_potential_df = Stage2Identifier(
    stock_name="ITC",
    extreme_points=filtered_extreme_points,
    price_jump=30,
    df=df_last_3_months
)

# Display the identified potential stage 2 moves
print("Identified Stage 2 Potential Moves:")
print(stage2_potential_df)

Starting Stage2Identifier for stock: ITC with price jump criterion: 30%
Analyzing from start point: 0 in extreme points.
supertrend called from low_point 4012 and high_point is 4033
Initial noise period index: 4016.
output_json : {'superUpTrend': False, 'price_jump': 16.864860805514343, 'time_taken': 21, 'slope': 0.8030886097863973, 'desc': 'Price jump < 30%. ', 'max_1D_decline': -5.025870582690624, 'max_5D_decline': -8.415843154762074}
supertrend called from low_point 4012 and high_point is 4155
Initial noise period index: 4034.
Check 2 failed: MA order incorrect at index 4095.
output_json : {'superUpTrend': False, 'price_jump': 36.675672032625926, 'time_taken': 143, 'slope': 0.2564732309973841, 'desc': 'Moving average check is FAILED and time_taken is 143', 'max_1D_decline': -7.644439697265624, 'max_5D_decline': -8.525251908735795}
supertrend called from low_point 4012 and high_point is 4308
Initial noise period index: 4057.
Check 2 failed: MA order incorrect at index 4095.
output_js

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2201745387.py:90: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

output_json : {'superUpTrend': True, 'price_jump': 36.922038871363604, 'time_taken': 147, 'slope': 0.2511703324582558, 'desc': '', 'max_1D_decline': -21.17036607659825, 'max_5D_decline': -22.624276447129862}
Super uptrend confirmed.
New record inserted   Stock_name  Start_Index  End_Index  Price_Percentage_Change    Slope  \
0        ITC         4161       4308                36.922039  0.25117   

   Change_Time_Taken Start_date   End_Date  max_1D_decline  max_5D_decline  \
0                147 2012-05-08 2012-12-12      -21.170366      -22.624276   

   current_high_point  
0                   6  
Initial noise period index: 4199.
output_json : {'superUpTrend': True, 'price_jump': 58.2086216211684, 'time_taken': 248, 'slope': 0.23471218395632418, 'desc': '', 'max_1D_decline': -21.17036607659825, 'max_5D_decline': -22.624276447129862}
Super uptrend confirmed.
New record inserted   Stock_name  Start_Index  End_Index  Price_Percentage_Change     Slope  \
0        ITC         4161       

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2201745387.py:90: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

output_json : {'superUpTrend': False, 'price_jump': 13.77910032929125, 'time_taken': 33, 'slope': 0.41754849482700757, 'desc': 'Price jump < 30%. Moving average check is FAILED and time_taken is 33', 'max_1D_decline': -10.057580402396233, 'max_5D_decline': -9.282944553582244}
supertrend called from low_point 6092 and high_point is 6097
Check 1 failed: 200-day MA not increasing at index 6093. start_index @6092 & end_index @6097
output_json : {'superUpTrend': False, 'price_jump': 31.12926597065733, 'time_taken': 5, 'slope': 6.225853194131465, 'desc': 'Time taken 5 not in range 10 & None. Moving average check is FAILED and time_taken is 5', 'max_1D_decline': -17.988668555240793, 'max_5D_decline': -17.70538243626062}
supertrend called from low_point 6092 and high_point is 6111
Check 1 failed: 200-day MA not increasing at index 6093. start_index @6092 & end_index @6111
output_json : {'superUpTrend': False, 'price_jump': 43.94501576099069, 'time_taken': 19, 'slope': 2.3128955663679314, 'desc

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\1206315221.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2201745387.py:90: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [ ]:
stage2_potential_df

,Stock_name,Start_Index,End_Index,Price_Percentage_Change,Slope,Change_Time_Taken,Start_date,End_Date,max_1D_decline,max_5D_decline,current_high_point
0,ITC,4161,4308,36.922039,0.251170,147,2012-05-08,2012-12-12,-21.170366,-22.624276,6
1,ITC,4161,4409,58.208622,0.234712,248,2012-05-08,2013-05-10,-21.170366,-22.624276,8
2,ITC,4161,4462,69.756525,0.231749,301,2012-05-08,2013-07-24,-21.170366,-22.624276,10
3,ITC,5343,5428,37.599371,0.442346,85,2017-02-27,2017-07-03,-12.556367,-12.344980,30
4,ITC,6578,6926,141.400972,0.406325,348,2022-02-24,2023-07-24,-14.452822,-14.274548,60


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize the figure with subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=('ITC.NS Historical Data', 'Volume'),
                    row_heights=[0.8, 0.2], specs=[[{"secondary_y": False}], [{"secondary_y": True}]])

# Define colors for increasing and decreasing candles
increasing_color = 'green'
decreasing_color = 'red'

# Add the candlestick chart to the first subplot with matching outline and fill colors
fig.add_trace(go.Candlestick(x=df_last_3_months['Date'],
                             open=df_last_3_months['Open'],
                             high=df_last_3_months['High'],
                             low=df_last_3_months['Low'],
                             close=df_last_3_months['Close'],
                             increasing=dict(line=dict(color=increasing_color, width=1), fillcolor=increasing_color),
                             decreasing=dict(line=dict(color=decreasing_color, width=1), fillcolor=decreasing_color),
                             name="Candlestick"), row=1, col=1)

# Add moving averages
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA200'],
                         mode='lines', name='MA200',
                         line=dict(width=1, color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA150'],
                         mode='lines', name='MA150',
                         line=dict(width=1, color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA50'],
                         mode='lines', name='MA50',
                         line=dict(width=1, color='green')), row=1, col=1)

# Iterate through stage2_potential_df to add trend lines and markers
for _, row in stage2_potential_df.iterrows():
    start_date = row['Start_date']
    end_date = row['End_Date']
    start_index = row['Start_Index']
    end_index = row['End_Index']

    # Find start and end values using the index in df_last_3_months
    start_value = df_last_3_months.loc[df_last_3_months.index == start_index, 'Low'].values[0]
    end_value = df_last_3_months.loc[df_last_3_months.index == end_index, 'High'].values[0]

    # Add a line trace for this trend
    fig.add_trace(go.Scatter(x=[start_date, end_date], y=[start_value, end_value],
                             mode='lines', name=f"Trend: {start_date} to {end_date}",
                             line=dict(width=2, dash='dot')), row=1, col=1)

    # Add markers for stage 2 start and end
    fig.add_trace(go.Scatter(x=[start_date], y=[start_value],
                             mode='markers+text', name='Stage 2 Start',
                             marker=dict(color='Red', size=10),
                             text=["Stage 2 Start"],
                             textposition="bottom center"), row=1, col=1)

    fig.add_trace(go.Scatter(x=[end_date], y=[end_value],
                             mode='markers+text', name='Stage 2 End',
                             marker=dict(color='Red', size=10),
                             text=["Stage 2 End"],
                             textposition="top center"), row=1, col=1)

# Add volume trace to the second subplot
colors = ['green' if close >= open_ else 'red' for open_, close in zip(df_last_3_months['Open'], df_last_3_months['Close'])]
fig.add_trace(go.Bar(x=df_last_3_months['Date'], y=df_last_3_months['Volume'], marker_color=colors, name='Volume'), row=2, col=1)

# Customize layout without setting xaxis_type to 'category'
fig.update_layout(height=600, title='ITC.NS Historical Data (Last 3 Months) with Moving Averages, Trend Lines, and Volume',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False,
                  showlegend=False)

# Show the plot
fig.show()


# **Contraction**
---

#### **VCP_Correction_Magnitude_Check**

In [ ]:
def VCP_Correction_Magnitude_Check(start_point, extreme_points):

    start_point = 2
    length_of_previous_trend_move = 0
    uptrend_correction_threshold = 40  # Example threshold value
    uptrend_retest_boundary_threshold = 0  # Example threshold value

    small_check_trend_status, small_check_no_of_skips = Uptrend_BREATHER_IDENTIFICATION(
        start_point,
        filtered_extreme_points,
        length_of_previous_trend_move,
        uptrend_correction_threshold= 10,
        uptrend_retest_boundary_threshold = 0.2)

    large_check_trend_status, large_check_no_of_skips = Uptrend_BREATHER_IDENTIFICATION(
        start_point,
        filtered_extreme_points,
        length_of_previous_trend_move,
        uptrend_correction_threshold= 40,
        uptrend_retest_boundary_threshold = 0.2)

    # print(f"Smaller check trend status: {small_check_trend_status}, Number of skips: {small_check_no_of_skips}")
    # print(f"Large check trend status: {large_check_trend_status}, Number of skips: {large_check_no_of_skips}")

    if small_check_trend_status == 'Trend Reversed' :
        if large_check_trend_status == 'Breather Continued without trend reversal or continuation' or large_check_trend_status == 'Trend Continued':
            return True

    return False

In [ ]:
VCP_Correction_Magnitude_Check_Flag = VCP_Correction_Magnitude_Check(start_point, filtered_extreme_points)
print(f"Outcone of VCP_Correction_Magnitude_Check : {VCP_Correction_Magnitude_Check_Flag}")

Outcone of VCP_Correction_Magnitude_Check : True


#### **VCP_Zig_Zag**

In [ ]:
def VCP_zig_zag(start_candle_index, df, percent_threshold=0,  move_relative_percent_threshold=0, absolute_threshold=0):
    """
    Generates a Zig Zag pattern by identifying a series of high and low extreme points
    in a given OHLC DataFrame.

    Parameters:
    - df: DataFrame containing OHLC data.
    - percent_threshold: The percentage movement threshold for considering an extreme point.
    - absolute_threshold: The absolute movement threshold for considering an extreme point.

    Returns:
    - A list of tuples, each containing the index of the extreme point and its type ("MAX" or "MIN").
    """
    percent_drop = 0

    # For first contraction we need to ensure percentage_threshold is set to 9.99 for else not needed
    T1_flag = True

    # Ensure the 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Start with the first candle in the DataFrame
    if start_candle_index is None:
        start_candle_index = df.index.min()

    # Initialize the list to store the extreme points
    extreme_points = []
    start_candle_value = df.loc[start_candle_index,'High']
    # Inserting the first point since it is the first extreme point
    extreme_points.append((start_candle_index, start_candle_value, 'High'))

    current_type = None  # Keep track of the current extreme type to alternate between high and low

    while True:
        # if T1_flag:
        #     print(f"percent_threshold >>>>> {percent_threshold}")
        next_point_index, next_extreme_value, extreme_type = next_extreme_candle(df, start_candle_index, percent_threshold=percent_threshold, move_relative_percent_threshold=move_relative_percent_threshold,  absolute_threshold=absolute_threshold)
        print(f"extreme_type is {extreme_type}")
        if extreme_type == 'HIGH':
            percent_drop = (df.loc[next_point_index,'High'] -  df.loc[start_candle_index,'Low']) / df.loc[start_candle_index,'Low'] * 100
        elif extreme_type == 'LOW':
            percent_drop = (df.loc[start_candle_index,'High'] -  df.loc[next_point_index,'Low']) / df.loc[start_candle_index,'High'] * 100
        print(f"percent_threshold >>>>> {percent_threshold} || percent_drop is {percent_drop }")
        percent_threshold = 0.40 * max(percent_drop, percent_threshold)
        print(f"new percent_threshold >>>>> {percent_threshold} ")

        # Check if the next_point_index is None or if we've encountered a repeat
        if next_point_index == start_candle_index :
            # print("Encountered a repeat index")
            # Repeat index happens because that point itself is the extreme point hence it points at itsel
            if next_point_index is None:
                # print("Reached the End of dataframe")
                2==2
            break

        # Add the found extreme point to the list
        extreme_points.append((next_point_index, next_extreme_value, extreme_type))

        # Update start_candle_index for the next iteration
        start_candle_index = next_point_index

        # Check if we've reached the end of the DataFrame
        if start_candle_index == df.index.max():
            # print("Reached the end of DataFrame.")
            break

    return extreme_points


In [ ]:
start_index = get_index('2023-07-24', df_last_3_months)

print(f"start_index is {start_index}")
vcp_extreme_points = VCP_zig_zag(
    start_index,
    df_last_3_months,
    percent_threshold = 10,
    move_relative_percent_threshold = 40,
    absolute_threshold = 0)

# vcp_extreme_points = zig_zag(
#     df_last_3_months,
#     percent_threshold = 10,
#     move_relative_percent_threshold = 0,
#     absolute_threshold = 0)

start_index is 6926
New pivot low found: 436.1000061035156 at index 6953
New pivot low found: 433.2999877929688 at index 6975
New pivot low found: 433.1000061035156 at index 6988
New pivot low found: 427.0499877929688 at index 6989
New pivot low found: 425.5 at index 6994
Threshold met. Returning pivot low index: 6994
extreme_type is LOW
percent_threshold >>>>> 10 || percent_drop is 14.848911425739452
new percent_threshold >>>>> 5.939564570295781 
New pivot high found: 457.7999877929688 at index 7016
New pivot high found: 464.5 at index 7017
New pivot high found: 464.7999877929688 at index 7018
New pivot high found: 467.8999938964844 at index 7033
New pivot high found: 469.9500122070313 at index 7034
New pivot high found: 471.3500061035156 at index 7035
New pivot high found: 480.7000122070313 at index 7036
New pivot high found: 481.4500122070313 at index 7037
Threshold met. Returning pivot high index: 7037
extreme_type is HIGH
percent_threshold >>>>> 5.939564570295781 || percent_drop i

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\3646839889.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ayann\AppData\Local\Temp\ipykernel_8624\2429544573.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
vcp_extreme_points

[(6926, 499.7000122070313, 'High'),
 (6994, 425.5, 'LOW'),
 (7037, 481.4500122070313, 'HIGH'),
 (7061, 408.6000061035156, 'LOW')]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Initialize the figure with subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, subplot_titles=('ITC.NS Historical Data', 'Volume'),
                    row_heights=[0.8, 0.2], specs=[[{"secondary_y": False}], [{"secondary_y": True}]])

# Define colors for increasing and decreasing candles
increasing_color = 'green'
decreasing_color = 'red'

# Add the candlestick chart to the first subplot with matching outline and fill colors
fig.add_trace(go.Candlestick(x=df_last_3_months['Date'],
                             open=df_last_3_months['Open'],
                             high=df_last_3_months['High'],
                             low=df_last_3_months['Low'],
                             close=df_last_3_months['Close'],
                             increasing=dict(line=dict(color=increasing_color, width=1), fillcolor=increasing_color),
                             decreasing=dict(line=dict(color=decreasing_color, width=1), fillcolor=decreasing_color),
                             name="Candlestick"), row=1, col=1)

# Function to extract date and value for plotting
def extract_date_value(points, df):
    # Corrected function to ensure 'idx' is defined in both comprehensions
    dates = [df.loc[idx, 'Date'] for idx, value, point_type in points if idx in df.index]
    values = [value for idx, value, point_type in points if idx in df.index]
    return dates, values

# VCP Extreme Points (with trend lines)
dates, values = extract_date_value(vcp_extreme_points, df_last_3_months)
fig.add_trace(go.Scatter(x=dates, y=values, mode='lines+markers',
                         line=dict(color='Blue', width=2),
                         marker=dict(color='LightSkyBlue', size=1),
                         name='Original Trend'))


# Add moving averages
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA200'],
                         mode='lines', name='MA200',
                         line=dict(width=1, color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA150'],
                         mode='lines', name='MA150',
                         line=dict(width=1, color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_last_3_months['Date'], y=df_last_3_months['MA50'],
                         mode='lines', name='MA50',
                         line=dict(width=1, color='green')), row=1, col=1)

# Iterate through stage2_potential_df to add trend lines and markers
for _, row in stage2_potential_df.iterrows():
    start_date = row['Start_date']
    end_date = row['End_Date']
    start_index = row['Start_Index']
    end_index = row['End_Index']

    print(f"start_index >> {start_index}")

    # Find start and end values using the index in df_last_3_months
    start_value = df_last_3_months.loc[df_last_3_months.index == start_index, 'Low'].values[0]
    end_value = df_last_3_months.loc[df_last_3_months.index == end_index, 'High'].values[0]

    # Add a line trace for this trend
    fig.add_trace(go.Scatter(x=[start_date, end_date], y=[start_value, end_value],
                             mode='lines', name=f"Trend: {start_date} to {end_date}",
                             line=dict(color = 'Red', width=2, dash='dot')), row=1, col=1)

    # Add markers for stage 2 start and end
    fig.add_trace(go.Scatter(x=[start_date], y=[start_value],
                             mode='markers+text', name='Stage 2 Start',
                             marker=dict(color='Red', size=10),
                             text=["Stage 2 Start"],
                             textposition="bottom center"), row=1, col=1)

    fig.add_trace(go.Scatter(x=[end_date], y=[end_value],
                             mode='markers+text', name='Stage 2 End',
                             marker=dict(color='Red', size=10),
                             text=["Stage 2 End"],
                             textposition="top center"), row=1, col=1)

# Add volume trace to the second subplot
colors = ['green' if close >= open_ else 'red' for open_, close in zip(df_last_3_months['Open'], df_last_3_months['Close'])]
fig.add_trace(go.Bar(x=df_last_3_months['Date'], y=df_last_3_months['Volume'], marker_color=colors, name='Volume'), row=2, col=1)

# Customize layout without setting xaxis_type to 'category'
fig.update_layout(height=600, title='ITC.NS Historical Data (Last 3 Months) with Moving Averages, Trend Lines, and Volume',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False,
                  showlegend=False)

# Show the plot
fig.show()


start_index >> 4161
start_index >> 4161
start_index >> 4161
start_index >> 5343
start_index >> 6578
